In [1]:
from typing import List
from math import floor

In [2]:
s_example = "2333133121414131402"

with open('input.txt', 'r') as f:
    s_input = f.read()[:-1]

# Part 1

In [3]:
def construct(s: str) -> List[int]:
    arr = [int(c) for c in list(s)]
    res = [0 for _ in range(arr[0])]
    L = len(arr)
    left, right = 1, 2 * floor((L-1)/2)
    while left <= right:
        while arr[right] > 0 and arr[left] > 0: #while it is possible to transfer from idx right to empty space at idx left
            res.append(right//2) #add element to the final array
            arr[right] -= 1 #update number of elements to transfer
            arr[left] -= 1 #update number of free spaces remaining
        if arr[right] == 0:
            right -= 2
        if arr[left] == 0: #exhausted free spaces at idx left
            res += ((left+1)//2 for _ in range(arr[left+1])) #add elements corresponding to idx (left + 1) to final array
            left += 2
    return res

def checksum(s: str) -> int:
    arr = construct(s)
    return sum((i*el for (i,el) in enumerate(arr)))

In [4]:
checksum(s_example)

1928

In [5]:
checksum(s_input)

6446899523367

# Part 2

In [6]:
def init_idxs_leftmost_blocks(arr):
    l_max = max((el for (i, el) in enumerate(arr) if i%2 == 1))
    idxs_leftmost_blocks = [-1 for _ in range(l_max+1)]
    max_until_now = 0
    for (i, el) in enumerate(arr):
        if i%2 == 1 and el > max_until_now:
            for j in range(max_until_now+1, el+1):
                idxs_leftmost_blocks[j] = i
            max_until_now = el
    return l_max, idxs_leftmost_blocks

def update(arr, right):
    l_max = max((el for (i, el) in enumerate(arr) if i%2 == 1 and i < right))
    idxs_leftmost_blocks = [-1 for _ in range(l_max+1)]
    max_until_now = 0
    for (i, el) in enumerate(arr):
        if i%2 == 1 and i < right and el > max_until_now:
            for j in range(max_until_now+1, el+1):
                idxs_leftmost_blocks[j] = i
            max_until_now = el
    return l_max, idxs_leftmost_blocks

def construct_dic(s: str) -> List[int]:
    arr = [int(c) for c in list(s)]
    L = len(arr)
    #for each length l of free space that occurs, store the index of the rightmost free space of length >= l, and store the max of all the l's at all time
    l_max, idxs_leftmost_blocks = init_idxs_leftmost_blocks(arr)
    
    right = 2 * floor((L-1)/2)
    write_at_index = {0: [(0, arr[0])]} 
    #for each index of free space that will be overwritten, provide what to write, and how many occurrences
    #also remember static blocks for later
    
    while right > 0:
        if arr[right] <= l_max and idxs_leftmost_blocks[arr[right]] != -1:
            idx_leftmost_block = idxs_leftmost_blocks[arr[right]]
            write_at_index.setdefault(idx_leftmost_block, []).append((right//2, arr[right]))
            write_at_index.setdefault(right, []).append((0, arr[right]))
            arr[idx_leftmost_block] -= arr[right]
            l_max, idxs_leftmost_blocks = update(arr, right-2)
        else:
            write_at_index.setdefault(right, []).append((right//2, arr[right]))
        right -= 2

    for (i, el) in enumerate(arr):
        if i%2 == 1 and arr[i] > 0:
            write_at_index.setdefault(i, []).append((0, arr[i]))
    return write_at_index

def construct_from_dic(d):
    res = []
    for k in sorted(d.keys()):
        for (num, occ) in d[k]:
            res += [num for _ in range(occ)]
    return res
            
def checksum2(s: str) -> int:
    arr = construct_from_dic(construct_dic(s))
    return sum((i*el for (i,el) in enumerate(arr)))

In [7]:
checksum2(s_example)

2858

In [8]:
checksum2(s_input)

6478232802631